# Setup

In [1]:
from datetime import datetime
import io
import itertools
from packaging import version

import tensorflow as tf
from tensorflow.keras import (
    layers,
    datasets,
    Sequential,
    callbacks,
)

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

In [2]:
%load_ext tensorboard

# Download the Fashion-MNIST dataset

In [3]:
fashion_mnist = datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Visualizing a Single Image

In [4]:
print(f'Shape of the data : {train_images[0].shape}')
print(f'Label: {train_labels[0]} -> {class_names[train_labels[0]]}')

Shape of the data : (28, 28)
Label: 9 -> Ankle boot


In [5]:
img = np.reshape(train_images[0], (-1, 28, 28, 1))

In [6]:
# Clear any previous logs
!rm -rf logs

logdir = 'logs/train_data/' + datetime.now().strftime('%Y%m%d-%H%M%S')
file_writer = tf.summary.create_file_writer(logdir)

with file_writer.as_default():
    tf.summary.image('Training data', img, step=0)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-03-14 15:05:09.034491: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-14 15:05:09.034667: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Visualizing Multiple Images

In [7]:
with file_writer.as_default():
    images = np.reshape(train_images[75:100], (-1, 28, 28, 1))
    tf.summary.image('25 training images', images, max_outputs=25, step=0)

## Logging an Arbitrary Image Data

In [8]:
!rm -rf logs/plots

logdir = 'logs/plots/' + datetime.now().strftime('%Y%m%d-%H%M%S')
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
    
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    
    plt.close(figure)
    buf.seek(0)
    
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

def image_grid():
    figure = plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_images[i], cmap=plt.cm.binary)
        
    return figure

figure = image_grid()

with file_writer.as_default():
    tf.summary.image('Training data', plot_to_image(figure), step=0)
    
%tensorboard --logdir logs/plots

# Building an Image Classifier

In [9]:
model = Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix

    Args:
        cm (array, shape=[n, n]): A confusion matrix of integer classes
        class_names (array, shape=[n]): String names of the integer classes
    """
    
    figure = plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Compute the labels from the normalized confusion matrix
    labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if the label is dark, otherwise use black text
    threshold = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = 'white' if cm[i, j] > threshold else 'black'
        plt.text(j, i, labels[i, j], horizontalalignment='center', color= color)
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicter label')
    return figure

In [11]:
# Remove any previous logs
!rm -rf logs/image

logdir = 'logs/image/' + datetime.now().strftime('%Y%m%d-%H%M%S')

tensorboard_cb = callbacks.TensorBoard(log_dir=logdir)
file_writer = tf.summary.create_file_writer(logdir + '/cm')

In [12]:
def log_confusion_matrix(epoch, logs):
    
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)
    
    # Calculate the confusion matrix
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)
    
    with file_writer.as_default():
        tf.summary.image('Confusion Matrix', cm_image, step=epoch)
    
cm_cb = callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)


In [13]:
%tensorboard --logdir logs/image

model.fit(
    train_images,
    train_labels,
    epochs=10,
    verbose=0,
    callbacks=[tensorboard_cb, cm_cb],
    validation_data=[test_images, test_labels]
)

2022-03-14 15:05:14.039986: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-14 15:05:14.185947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 15:05:22.420757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 15:05:23.564249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
